### Imports & Definitions

In [1]:
import sys
sys.path.append("../../utils")
from definitions import *
from path_helpers import get_dataset_path, get_metric_dir_path
from mera_helpers import construct_prompt
from huggingface_helpers import get_tokenizer, get_model
from llm_helpers import calculate_token_interest_probs, get_answer

### Learn to load datasets from disk

In [2]:
for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
    for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
        path = get_dataset_path(subset, name, split)
        dataset = load_from_disk(path)
        print(subset, dataset[0])

rummlu {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
ruopenbookqa {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A

In [3]:
rummlu = {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
construct_prompt(rummlu)

'Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.\nВ какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.\nA Нарушает, Нарушает\nB Нарушает, Не нарушает\nC Не нарушает, Нарушает\nD Не нарушает, Не нарушает\nЗапишите букву правильного ответа\nОтвет:'

In [4]:
ruopenbookqa = {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: ', 'inputs': {'question': 'Примером потомства, получившего генетический набор от родителей, является', 'option_a': 'маленький кротенок', 'option_b': 'маленький мешок сахара', 'option_c': 'маленькая тележка', 'option_d': 'маленький пирожок'}, 'outputs': 'A', 'meta': {'id': 0}}
construct_prompt(ruopenbookqa)

'Примером потомства, получившего генетический набор от родителей, является\nA. маленький кротенок\nB. маленький мешок сахара\nC. маленькая тележка\nD. маленький пирожок\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: '

### Learn to inference LLM on these datasets

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    tokenizer = get_tokenizer(baseline_model, HUGGINGFACE_MODEL_TO_REPO[baseline_model])
    model = get_model(baseline_model, HUGGINGFACE_MODEL_TO_REPO[baseline_model])
    model.to(device)
    q = construct_prompt(rummlu)
    print(q)
    probs = calculate_token_interest_probs(q, tokenizer, model)
    print(probs)
    a = calculate_probs_and_get_answer(q, tokenizer, model)
    print(a)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:
{'A': 0.08026155829429626, 'B': 0.09532341361045837, 'C': 0.09801381826400757, 'D': 0.07560519129037857}
C


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:
{'A': 0.15552033483982086, 'B': 0.18379555642604828, 'C': 0.12278010696172714, 'D': 0.12691020965576172}
B


### Inference LLMs on all datasets

In [5]:
# add tqdm
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    repo = HUGGINGFACE_MODEL_TO_REPO[baseline_model]
    tokenizer = get_tokenizer(baseline_model, repo)
    model = get_model(baseline_model, repo)
    model.to(device)
    for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
        for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
            path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(path)
            probs_list = []
            a_list = []
            for row in dataset:
                q = construct_prompt(row)
                probs = calculate_token_interest_probs(q, tokenizer, model)
                probs_list.append({
                    "probs": probs,
                    "meta": row["meta"],
                })
                a = get_answer(probs)
                a_list.append({
                    "answer": a,
                    "meta": row["meta"],
                })
            metric_dir_path = get_metric_dir_path(baseline_model, subset, name, split)
            metric_dir_path.mkdir(exist_ok=True, parents=True)
            with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
                json.dump(probs_list, f, ensure_ascii=False, indent=2)
            with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
                json.dump(a_list, f, ensure_ascii=False, indent=2)